In [1]:
import re
import glob
from pathlib import Path

import pandas as pd


In [2]:
ESTAB_COL_INDEX = {"cnpj_basico": 0, "nome_fantasia": 4, "uf": 19, "codigo_municipio": 20}
SOC_COL_INDEX = {
    "cnpj_basico": 0,
    "nome_socio_razao_social": 2,
    "cpf_cnpj_socio": 3,
    "qualificacao_socio": 4,
    "data_entrada_sociedade": 5,
}

ESTAB_COLS = ["cnpj_basico", "nome_fantasia", "uf", "codigo_municipio"]
SOC_COLS = ["cnpj_basico", "nome_socio_razao_social", "cpf_cnpj_socio", "qualificacao_socio", "data_entrada_sociedade"]

FINAL_COLS = [
    "cnpj_basico",
    "nome_fantasia",
    "uf",
    "codigo_municipio",
    "nome_socio_razao_social",
    "cpf_cnpj_socio",
    "qualificacao_socio",
    "data_entrada_sociedade",
]


In [3]:
def _digits(s: str) -> str:
    return re.sub(r"\D", "", s or "")

def _cnpj_basico(cnpj: str) -> str:
    d = _digits(cnpj)
    if len(d) < 8:
        raise ValueError("CNPJ inválido (mínimo 8 dígitos).")
    return d[:8]

def _find_base_dir() -> Path:
    p = Path("./dados_extraidos")
    if p.exists() and p.is_dir():
        return p
    raise FileNotFoundError("Não encontrei a pasta ./dados_extraidos")

def _list_files(base_dir: Path, prefix: str) -> list[Path]:
    files = []
    for i in range(10):
        files.extend(Path(p) for p in glob.glob(str(base_dir / f"{prefix}{i}" / "**" / "*"), recursive=True))
    return sorted({p for p in files if p.is_file()})


In [4]:
def _read_filtered_text(
    files: list[Path],
    cols: list[str],
    col_index: dict[str, int],
    cnpj_basico_alvo: str,
    chunksize: int = 600_000,
) -> pd.DataFrame:
    usecols = [col_index[c] for c in cols]
    frames: list[pd.DataFrame] = []

    for fp in files:
        if fp.suffix.lower() in {".parquet", ".pq"}:
            continue

        it = pd.read_csv(
            fp,
            sep=";",
            encoding="latin1",
            header=None,
            usecols=usecols,
            dtype="string",
            chunksize=chunksize,
            low_memory=False,
            memory_map=True,
        )

        for chunk in it:
            chunk.columns = cols
            chunk["cnpj_basico"] = (
                chunk["cnpj_basico"]
                .fillna("")
                .astype("string")
                .str.replace(r"\D", "", regex=True)
                .str[:8]
            )
            chunk = chunk[chunk["cnpj_basico"] == cnpj_basico_alvo]
            if not chunk.empty:
                frames.append(chunk)

    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=cols)


In [5]:
def buscar_por_cnpj(cnpj_alvo: str, chunksize: int = 600_000) -> pd.DataFrame:
    cnpj_basico_alvo = _cnpj_basico(cnpj_alvo)
    base_dir = _find_base_dir()

    est_files = _list_files(base_dir, "Estabelecimentos")
    soc_files = _list_files(base_dir, "Socios")

    df_est = _read_filtered_text(est_files, ESTAB_COLS, ESTAB_COL_INDEX, cnpj_basico_alvo, chunksize=chunksize)
    df_soc = _read_filtered_text(soc_files, SOC_COLS, SOC_COL_INDEX, cnpj_basico_alvo, chunksize=chunksize)

    if df_est.empty or df_soc.empty:
        return pd.DataFrame(columns=FINAL_COLS)

    return df_est.merge(df_soc, on="cnpj_basico", how="inner")[FINAL_COLS]


In [ ]:
CNPJ_ALVO = "07397020" 
df = buscar_por_cnpj(CNPJ_ALVO)
df


,cnpj_basico,nome_fantasia,uf,codigo_municipio,nome_socio_razao_social,cpf_cnpj_socio,qualificacao_socio,data_entrada_sociedade
0,07397020,BRASIL SHITAQUE,SP,6493,SANDRA APARECIDA DA SILVA,***887432**,22,20050520
1,07397020,BRASIL SHITAQUE,SP,6493,SILVIO GUIMARAES CASTRO SOBRINHO,***227288**,49,20050520
2,07397020,BRASIL SHITAQUE,SP,6493,SANDRA APARECIDA DA SILVA,***887432**,22,20050520
3,07397020,BRASIL SHITAQUE,SP,6493,SILVIO GUIMARAES CASTRO SOBRINHO,***227288**,49,20050520


In [ ]:
saida = f"resultado_{_cnpj_basico(CNPJ_ALVO)}.xlsx"
df.to_excel(saida, index=False)
saida #exemplo 1 tempo de carregamento : 7m


'resultado_41482239.xlsx'

In [ ]:
saida = f"resultado_{_cnpj_basico(CNPJ_ALVO)}.xlsx"
df.to_excel(saida, index=False)
saida #exemplo2 tempo de carregamento : 7m

'resultado_07397020.xlsx'